In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

True

In [18]:
llm=HuggingFaceEndpoint(repo_id="meta-llama/Llama-4-Scout-17B-16E-Instruct",task='text-generation')
enhancer_model=ChatHuggingFace(llm=llm)

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\panka\langchain\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\panka\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [44]:
chat_template=ChatPromptTemplate(
    [
        ("system","""You are a precise writing enhancement assistant. Your ONLY job is to improve the expression of text — not its ideas.
You MUST:
- Fix grammatical errors and spelling mistakes
- Improve sentence structure and clarity
- Improve logical flow between sentences
- Keep the improved text approximately the same length as the input

You MUST NOT:
- Add any new ideas, arguments, or information not present in the original
- Remove any core meaning or factual claim from the original
- Change the factual intent of any sentence
- Expand content beyond style and grammatical improvement
- Add preambles like "Here is the improved text" — return ONLY the enhanced text
- Ask any further questions to user just enhance the text and return it"""),
        ("human","""Below are examples of how to enhance text:
Input: I am doing project on AI which is very good and it help many people in future.
Output: I am working on an AI project that has strong potential to help many people in the future.

Now enchance the following text

Input: {prompt}
Output:""")
    ]
)

In [45]:
res=embedding_model.embed_query("I am good boy")

In [46]:
parser=StrOutputParser()

In [47]:
import numpy as np

def cosine_similarity(arr1,arr2):
    arr1=np.array(arr1)
    arr2=np.array(arr2)
    num=(arr1*arr2).sum()
    den=np.sqrt((arr1**2).sum())*np.sqrt((arr2**2).sum())
    return num/den

In [48]:
enhanced_chain=RunnableParallel(
    {
        "orignal_text":RunnablePassthrough(),
        "enhanced_text": chat_template|enhancer_model|parser
    }
)
parl_chain= RunnableParallel({
    "enhanced_text":RunnableLambda(lambda x:x['enhanced_text']),
    "orignal_embedding":RunnableLambda(lambda x:embedding_model.embed_query(x['orignal_text']['prompt'])),
    "enhanced_embedding":RunnableLambda(lambda x:embedding_model.embed_query(x['enhanced_text']))
})
parl_chain_2= RunnableParallel({
    "enhanced_text":RunnableLambda(lambda x:x['enhanced_text']),
    "similarity": RunnableLambda(lambda x:cosine_similarity(x['orignal_embedding'],x['enhanced_embedding']))
})

final_runnable=enhanced_chain | parl_chain | parl_chain_2

In [51]:
res=final_runnable.invoke({"prompt":"I am an ai engineer"})

In [52]:
res

{'enhanced_text': 'I am an artificial intelligence engineer.',
 'similarity': np.float64(0.8844137603967079)}

In [54]:
res=final_runnable.invoke({"prompt":"The medicine should not be given to children under 5 and pregnant women must avoid it completely."})
res

{'enhanced_text': 'The medicine should not be administered to children under 5, and pregnant women must avoid it entirely.',
 'similarity': np.float64(0.9781912909849053)}

In [55]:
res=final_runnable.invoke({"prompt":"He are a good boy."})
res

{'enhanced_text': 'He is a good boy.',
 'similarity': np.float64(0.975991228455573)}